In [1]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import os
import re
from shapely.geometry import Polygon, MultiPolygon
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [2]:
# Create a shapeName column containing the names of each Insurance Unit in lowercase and stripped
insurance_names = pd.read_excel(r'C:\Users\mieke\Documents\Msc Thesis\Datasets\Yield data\Karnataka\insurance_names.xlsx', index_col = 0)
insurance_names['shapeName'] = insurance_names['Insurance Unit'].copy()
insurance_names.shapeName = insurance_names.shapeName.str.lower()
insurance_names.shapeName = insurance_names.shapeName.str.strip()

In [4]:
# Import files related to yields and insurance units
yields = pd.read_excel(r'C:\Users\mieke\Documents\Msc Thesis\Datasets\Yield data\Karnataka\yields.xlsx', index_col = 0)

In [5]:
bound_types = ['gp', 'h'] # Different bound types
column_names = ['KGISGPName', 'KGISHobliN'] # Column names containing the name of the Insurance Unit
path_types = ['C:/Users/mieke/Documents/Msc Thesis/Datasets/Shapefiles/Karnataka_gp_shp/', 'C:/Users/mieke/Documents/Msc Thesis/Datasets/Shapefiles/Karnataka_h_shp/'] # Different paths
non_indices = [[28], [28, 10]] # District names which do not occur in the yields dataframe (descending order is important) (eg 28:vijayanagara)

In [ ]:
# This cell combines insurance unit names as given in the yields data with insurance unit names as given in the boundary data for hobli as well as gram panchayat

# Create empty dataframes
df_yields_geometry = pd.DataFrame() 
df_geometry = pd.DataFrame()

# Loop over each of the bound types separately
for t in range(len(bound_types)):
    df_part_names = insurance_names[insurance_names['Gram Panchayat/Hobli'] == bound_types[t]] # create df including the rows within insurance names df belonging to the current bound type 
    df_part_names = df_part_names[['shapeName', 'Insurance Unit', 'District']].copy() # select the relevant columns
    df_part_names = df_part_names[['shapeName', 'Insurance Unit', 'District']].drop_duplicates(ignore_index=True) # drop duplicates

    part_names = df_part_names['shapeName'].unique() # list of all unique shapenames within the dataframe with current bound type
    part_names = sorted(part_names) # sort the shapenames in ascending order

    path = path_types[t] # Set path
    folder_names = os.listdir(path) # contains all foldernames within the folder

    district_names = [] # create list to include all unique district names within the bounds geodatframe
    df_total = pd.DataFrame() # create empty dataframe
    # This for loop creates one big geodataframe out of all separate district wise geodataframes
    for i in folder_names:
        input_shp = gpd.read_file(path + i + '/' + i + '.shp')
        district_name = i[3:].lower() # district name as used within the bounds geodataframe
        district_names.append(district_name)
        input_shp['District'] = district_name
        df_total = pd.concat([df_total, input_shp], ignore_index=True)

    district_names = sorted(district_names) # sort district names in ascending order
    for k in non_indices[t]:
        district_names.pop(k) # drop all district names which are not in the yields dataframe for the current bound type

    # select the relevant columns of the geometry bounds geodataframe and sort them based on their Insurance Unit name
    insurance_bounds = df_total[[column_names[t], 'District', 'geometry']].sort_values([column_names[t], 'District'], ascending=True, ignore_index=True)
    # Create a shapeName column containing the names of each bound in lowercase and stripped
    insurance_bounds['shapeName'] = insurance_bounds[column_names[t]]
    insurance_bounds['shapeName'] = insurance_bounds.shapeName.str.lower()
    insurance_bounds['shapeName'] = insurance_bounds.shapeName.str.strip()

    df_part_geometry = insurance_bounds[['shapeName', column_names[t], 'District', 'geometry']].copy() # copy the previously created geodataframe
    df_part_geometry = df_part_geometry.drop_duplicates(subset = ['shapeName', column_names[t], 'District'], ignore_index=True) # drop duplicates if they have the same bound name and district
    df_part_geometry = df_part_geometry.sort_values(['shapeName'], ascending=True, ignore_index=True) # sort the geodataframe based on shapeName
    part_bounds = df_part_geometry['shapeName'].unique() # list of all unique shapenames within the bounds geodataframe with current bound type

    unique_districts = df_part_names['District'].unique() # list of all unique district names within the yields dataframe
    unique_districts = sorted(unique_districts) # sort the district names in ascending order

    # Make sure that the district names within the yields dataframe are equal to the spelling of the district names within the bounds geodataframe
    for i in range(len(unique_districts)):
        df_part_names['District'] = np.where(df_part_names['District'] == unique_districts[i], district_names[i], df_part_names['District'])

    # Create new columns
    df_part_names['Insurance_Bound'] = np.nan # 
    df_part_names['Insurance_Bound_f'] = np.nan #
    df_part_names['Similarity'] = np.nan # Highest similarity score
    df_part_names['geometry'] = np.nan # geometry

    # This loop finds the shapeName within the bounds geodataframe which is most similar to the shapeName within the yields dataframe
    for d in district_names:
        df_bounds = df_part_geometry[df_part_geometry['District'] == d].copy() # contains all rows for district d (bounds file)
        df_names = df_part_names[df_part_names['District'] == d].copy() # contains all rows for district d (yields file)
        # For every shapeName within the yields file, we check which shapeName within the bounds file is most similar to it
        # Note that it only considers the names within the same district, since sometimes a name is used multiple times
        for i in df_names.index:
            indices = []
            for j in df_bounds.index:
                indices.append(fuzz.token_set_ratio(df_names.loc[i, 'shapeName'],df_bounds.loc[j, 'shapeName'])) # in this case, better than sort_ratio
            df_part_names.loc[i,'Insurance_Bound'] = df_bounds.loc[df_bounds.index[np.argmax(indices)],column_names[t]] # the original Insurance Name corresponding to the highest similarity bound
            df_part_names.loc[i,'Insurance_Bound_f'] = df_bounds.loc[df_bounds.index[np.argmax(indices)],'shapeName'] # the shapeName corresponding to the highest similarity bound
            df_part_names.loc[i,'Similarity'] = np.max(indices) # the highest similarity found for the current insurance name
            df_part_names.loc[i, 'geometry'] = df_bounds.loc[df_bounds.index[np.argmax(indices)],'geometry'] # geometry corresponding to the highest similarity bound

    df_final = df_part_names[df_part_names['Similarity'] == 100] # only keep the Insurance Units for which we have derived a 100% similarity

    ## YIELDS PART
    yields_part = yields[yields['Gram Panchayat/Hobli'] == bound_types[t]] # create df including the rows within yields df belonging to the current bound type 

    unique_districts = yields_part['District'].unique() # list of all unique district names within the yields dataframe
    unique_districts = sorted(unique_districts) # sort the district names in ascending order

    # Make sure that the district names within the yields dataframe are equal to the spelling of the district names within the bounds geodataframe
    for i in range(len(unique_districts)):
        yields_part['District'] = np.where(yields_part['District'] == unique_districts[i], district_names[i], yields_part['District'])

    # Merge the yields datafrme with created geometry bounds geodataframe for current bound type, based on Insurance Unit and District 
    merged_df = yields_part.merge(df_part_names, on=['Insurance Unit', 'District'])#, how='left')
    # mergedDf_drop = mergedDf.dropna(subset='geometry') # Note: note needed, as it is immediately done. It is needed, when we use how='left' above

    df_part = merged_df[['Year', 'Season', 'Insurance Unit', 'Gram Panchayat/Hobli', 'District', 'Taluk', 'Crop', 'IRR_RF', 'Average Yield(Kg/Ha)', 'geometry']].copy() # select the relevant columns
    # df_part['Gram Panchayat/Hobli'] = bound_types[t] # Note: note needed, as it is already added above 

    df_yields_geometry = pd.concat([df_yields_geometry, df_part], ignore_index=True) # add the yields_geometry results for the current bound type to the final dataframe

    df_part_names['Gram Panchayat/Hobli'] = bound_types[t] # Add current bound type to dataframe 
    df_geometry = pd.concat([df_geometry, df_part_names], ignore_index=True) # add the geometry results for the current bound type to the final dataframe 

In [11]:

# Transform the geometry and yields_geometry dataframes to geodataframes
# Then, save the geometry and yields_geometry geodataframes
gdf_yields_geometry = gpd.GeoDataFrame(df_yields_geometry, geometry = df_yields_geometry.geometry, crs = {'init': 'epsg:32643'}) # 32643 is the original crs (found by using .crs for one of the input shp files)
gdf_yields_geometry.geometry = gdf_yields_geometry.geometry.to_crs(epsg = 4326) # Set crs to 4326
gdf_yields_geometry.to_file(r'C:\Users\mieke\Documents\Msc Thesis\Datasets\Yield Data\Karnataka\yields_geometry.geojson', driver="GeoJSON") 

gdf_geometry = gpd.GeoDataFrame(df_geometry, geometry = df_geometry.geometry, crs = {'init': 'epsg:32643'}) # 32643 is the original crs (found by using .crs for one of the input shp files)
gdf_geometry.geometry = gdf_geometry.geometry.to_crs(epsg = 4326)
gdf_geometry.to_file(r'C:\Users\mieke\Documents\Msc Thesis\Datasets\Yield Data\Karnataka\geometry.geojson', driver="GeoJSON") 

c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\mieke\anaconda3\envs\thesis_base\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes